In [15]:
pwd

'/home/wsuser/work'

In [16]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_f3e9883c6fbb4c67933c887541a68670 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='c9yI3mycgqxsxnlNRLEmi77OIxJVvKhJh1IsfHa6xdTP',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_f3e9883c6fbb4c67933c887541a68670.get_object(Bucket='fertilizerrecommendationproject-donotdelete-pr-jlztryfyeggnok', Key='Fertilizers_Recommendation_ System_For_Disease_ Prediction.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [17]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [18]:
ls

'Dataset Plant Disease'/   fruit_classification.tgz


In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [20]:
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [21]:
pwd

'/home/wsuser/work'

In [22]:
x_train = train_datagen.flow_from_directory(r"/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/train",target_size = (64,64),batch_size=24,class_mode = 'categorical')
x_test = test_datagen.flow_from_directory(r"/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/test",target_size = (64,64),batch_size=24,class_mode = 'categorical')

Found 5384 images belonging to 6 classes.
Found 1686 images belonging to 6 classes.


In [23]:
x_train.class_indices

{'Apple___Black_rot': 0,
 'Apple___healthy': 1,
 'Corn_(maize)___Northern_Leaf_Blight': 2,
 'Corn_(maize)___healthy': 3,
 'Peach___Bacterial_spot': 4,
 'Peach___healthy': 5}

In [24]:
x_test.class_indices

{'Apple___Black_rot': 0,
 'Apple___healthy': 1,
 'Corn_(maize)___Northern_Leaf_Blight': 2,
 'Corn_(maize)___healthy': 3,
 'Peach___Bacterial_spot': 4,
 'Peach___healthy': 5}

In [25]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

In [26]:
model = Sequential()
model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

In [27]:
model.add(Dense(64 ,activation='relu'))
model.add(Dense(64,activation = 'relu'))


In [28]:
model.add(Dense(6,activation = 'softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 30752)             0         
                                                                 
 dense (Dense)               (None, 64)                1968192   
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                        

In [29]:
model.fit_generator(x_train,steps_per_epoch=len(x_train),validation_data=x_test,validation_steps=len(x_test),epochs=10)

/tmp/wsuser/ipykernel_255/1582812018.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=len(x_train),validation_data=x_test,validation_steps=len(x_test),epochs=10)


Epoch 1/10
225/225 [==============================] - 17s 72ms/step - loss: 0.6884 - accuracy: 0.7576 - val_loss: 0.3611 - val_accuracy: 0.8749
Epoch 2/10
225/225 [==============================] - 16s 69ms/step - loss: 0.3105 - accuracy: 0.8893 - val_loss: 0.2152 - val_accuracy: 0.9247
Epoch 3/10
225/225 [==============================] - 15s 68ms/step - loss: 0.2601 - accuracy: 0.9092 - val_loss: 0.1709 - val_accuracy: 0.9466
Epoch 4/10
225/225 [==============================] - 16s 70ms/step - loss: 0.2021 - accuracy: 0.9294 - val_loss: 0.1740 - val_accuracy: 0.9413
Epoch 5/10
225/225 [==============================] - 16s 69ms/step - loss: 0.1774 - accuracy: 0.9398 - val_loss: 0.1676 - val_accuracy: 0.9431
Epoch 6/10
225/225 [==============================] - 15s 68ms/step - loss: 0.1510 - accuracy: 0.9491 - val_loss: 0.1394 - val_accuracy: 0.9490
Epoch 7/10
225/225 [==============================] - 16s 69ms/step - loss: 0.1406 - accuracy: 0.9478 - val_loss: 0.1729 - val_accuracy:

In [30]:
ls

'Dataset Plant Disease'/   fruit_classification.tgz


In [31]:
model.save("fruit.h5")

In [32]:
ls

'Dataset Plant Disease'/   fruit_classification.tgz   fruit.h5


In [33]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model =load_model("fruit.h5")
img = image.load_img(r"/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/test/Apple___healthy/0e8c4b90-36d4-4b8c-a134-88c344e97ac2___RS_HL 5721.JPG",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
y = np.argmax(model.predict(x),axis=1)

index=['Apple___Black_rot',
 'Apple___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Peach___Bacterial_spot',
 'Peach___healthy']

index[y[0]]

'Apple___healthy'

In [34]:
#Vegetable disease prediction model deployment

In [35]:
pip install watson_machine_learning-client

Note: you may need to restart the kernel to use updated packages.


In [36]:
from ibm_watson_machine_learning import APIClient
wml_credentials= {
                    "url" : "https://eu-de.ml.cloud.ibm.com",
                    "apikey" : "iEj50semUUlguSe77KfSVrjmIY-vqw5eOvRyNpExKk-r"
                }

In [37]:
client = APIClient(wml_credentials)

In [38]:
def guid_from_space_name(client,fruit_disease_deploy):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == fruit_disease_deploy)['metadata']['id'])

In [39]:
space_uid = guid_from_space_name(client,'fruit_disease_deploy')
print(space_uid)

ed666a23-e999-454f-8992-d7a2403984f9


In [44]:
client.set.default_space(space_uid)

'SUCCESS'

In [67]:
pip install --upgrade pip

     |████████████████████████████████| 2.0 MB 29.6 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
Note: you may need to restart the kernel to use updated packages.


In [72]:
client.software_specifications.list(100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

In [73]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')

In [74]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [64]:
ls

'Dataset Plant Disease'/       fruit_classification.tgz
 fruit_classification123.tgz   fruit.h5


In [56]:
!tar -zvcf fruit_classification123.tgz fruit.h5

fruit.h5


In [60]:
ls

'Dataset Plant Disease'/       fruit_classification.tgz
 fruit_classification123.tgz   fruit.h5


In [75]:
model_details=client.repository.store_model(model='fruit_classification123.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN Model Building",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [76]:
model_id=client.repository.get_model_id(model_details)

In [77]:
model_id

'd5e1b2aa-f578-4237-b9e9-64deab109563'

In [78]:
client.repository.download("d5e1b2aa-f578-4237-b9e9-64deab109563","fruit_classification123.tar.gb")

Successfully saved model content to file: 'fruit_classification123.tar.gb'


'/home/wsuser/work/fruit_classification123.tar.gb'